In [ ]:
import json
import pandas as pd
from google.colab import drive

In [ ]:
# @title Mount google drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Copy data to local cache
!cp -r /gdrive/'Shared drives'/'Sunbird AI'/Projects/'GIZ Mini-grid Identification'/Code/villages.json .

In [ ]:
villages = pd.read_json('./villages.json')

In [ ]:
villages.shape

(411, 15)

In [ ]:
villages.head()

,ID,NES_category,district,latitude,longitude,max_num_buildings_in_1km_radius,mean_wind_speed,num_buildings,num_permanent_buildings,parish,power_demand_analysis,power_supply_analysis,subcounty,vegetation_percentile,village
0,5500895,Grid extension,Lamwo,3.808296,33.023490,984,1.0,9,0,Ngacino,[],[{'detail': 'The vegetation index for this vil...,Agoro,79,Special Area
1,5500896,Candidate for solar home systems,Lamwo,3.842983,33.037389,6,1.0,12,2,Ngacino,[{'detail': 'There are 6 buildings within a 1k...,[{'detail': 'The vegetation index for this vil...,Agoro,79,Grazing Land
2,5500897,Candidate for solar home systems,Lamwo,3.827202,33.011559,133,1.0,97,1,Ngacino,[{'detail': 'There are 133 buildings within a ...,[{'detail': 'The vegetation index for this vil...,Agoro,79,Irumo
3,5500898,Candidate for solar home systems,Lamwo,3.820075,33.020129,575,1.0,98,0,Ngacino,[],[{'detail': 'The vegetation index for this vil...,Agoro,79,Obere
4,5500899,Candidate for solar home systems,Lamwo,3.810693,33.031540,0,1.0,1,0,Ngacino,[{'detail': 'There are 0 buildings within a 1k...,[{'detail': 'The vegetation index for this vil...,Agoro,79,Kila


In [ ]:
villages['NES_category'].value_counts()

Candidate for solar home systems    212
Grid extension                      168
Candidate minigrid site              22
Existing minigrid site                9
Name: NES_category, dtype: int64

## Criteria for selecting 10 (loose)
1. Based on NES categorizaton - select 2 existing minigrids, 2 candidate minigrids, 3 grid extension, 3 solar home
2. Of 2 exisiting minigrids, select one with 1km buildings > 500 and no of permanent buildings > 10, the other site, the oppposite.
3. Of the candidate minigrids, select according to same criteria above
4. For grid extension - select 1st with 1km buildings > 500, no. permanent > 10, veg index < 50. For 2nd select reverse, for 3rd with 1km buildings < 500, no. permanent > 10
5. For solar home systems select 1st with vegetation percentile > 80, 2nd with no. permanent > 5, 3rd with no. permanent = 0


In [ ]:
villages.columns

Index(['ID', 'NES_category', 'district', 'latitude', 'longitude',
       'max_num_buildings_in_1km_radius', 'mean_wind_speed', 'num_buildings',
       'num_permanent_buildings', 'parish', 'power_demand_analysis',
       'power_supply_analysis', 'subcounty', 'vegetation_percentile',
       'village'],
      dtype='object')

## Existing minigrids

In [ ]:
minigrid_villages = villages.query("NES_category == 'Existing minigrid site' \
                                   and max_num_buildings_in_1km_radius > 500 \
                                   and num_permanent_buildings > 20")
len(minigrid_villages)

2

In [ ]:
minigrid_villages_2 = villages.query("NES_category == 'Existing minigrid site' \
                                   and max_num_buildings_in_1km_radius < 500 \
                                   and num_permanent_buildings > 0 ")
len(minigrid_villages_2)

1

## Candidate minigrids

In [ ]:
candidate_minigrid_villages = villages.query("NES_category == 'Candidate minigrid site' \
                                   and max_num_buildings_in_1km_radius > 500 \
                                   and num_permanent_buildings > 20")
len(candidate_minigrid_villages)

2

In [ ]:
candidate_minigrid_villages_2 = villages.query("NES_category == 'Candidate minigrid site' \
                                   and max_num_buildings_in_1km_radius < 500 \
                                   and num_permanent_buildings > 10")
len(candidate_minigrid_villages_2)

4

## Grid Extension

In [ ]:
candidate_grid_extension_villages = villages.query("NES_category == 'Grid extension' \
                                   and max_num_buildings_in_1km_radius > 500 \
                                   and num_permanent_buildings > 0 \
                                   and vegetation_percentile < 78")
len(candidate_grid_extension_villages)

1

In [ ]:
candidate_grid_extension_villages_2 = villages.query("NES_category == 'Grid extension' \
                                   and max_num_buildings_in_1km_radius < 500 \
                                   and num_permanent_buildings > 20 \
                                   and vegetation_percentile > 80")
len(candidate_grid_extension_villages_2)

3

## Solar Home systems

In [ ]:
solar_home_villages = villages.query("NES_category == 'Candidate for solar home systems' \
                                   and max_num_buildings_in_1km_radius > 500 \
                                   and num_permanent_buildings > 10 \
                                   and vegetation_percentile > 78")
len(solar_home_villages)

10

## Selection

In [ ]:
selected_villages = []

selected_villages = \
    minigrid_villages.sample()['village'].to_list() + \
    minigrid_villages_2.sample()['village'].to_list() + \
    candidate_minigrid_villages.sample()['village'].to_list() + \
    candidate_minigrid_villages_2.sample()['village'].to_list() + \
    candidate_grid_extension_villages.sample()['village'].to_list() + \
    candidate_grid_extension_villages_2.sample(2)['village'].to_list() + \
    solar_home_villages.sample(3)['village'].to_list()

In [ ]:
selected_villages

['Ngomoromo',
 'Oboko',
 'Apyetta South',
 'Pabwoch East',
 'Central',
 'Lugwar Central',
 'Labworoyeng',
 'Palaa West',
 'Ateda',
 'Beyogoya']